In [1]:
import os
import pandas as pd
import numpy as np
import datetime

if not os.path.exists('aircraft.csv'):
    !wget https://github.com/sudakov/lab_it/raw/master/aircraft.csv
if not os.path.exists('airports.csv'):
    !wget https://github.com/sudakov/lab_it/raw/master/airports.csv
if not os.path.exists('airlines.csv'):
    !wget https://github.com/sudakov/lab_it/raw/master/airlines.csv
if not os.path.exists('cargo transportation.csv'):
    !wget https://github.com/sudakov/lab_it/raw/master/cargo%20transportation.csv
if not os.path.exists('passenger transportation.csv'):
    !wget https://github.com/sudakov/lab_it/raw/master/passenger%20transportation.csv

In [2]:
aircraft_df = pd.read_csv('aircraft.csv', sep=';')
airports_df = pd.DataFrame(pd.read_csv('airports.csv', sep=',').values, 
                           columns=['аэропорт', 'документ', 'компания', 'категория'])
airlines_df = pd.read_csv('https://raw.githubusercontent.com/sudakov/lab_it/master/airlines.csv', 
                            names=['короткое название', "полное название", "аэропорт", "типы воздушных судов"])
cargo_df = pd.read_csv('cargo transportation.csv', sep=';')
passenger_df = pd.read_csv('passenger transportation.csv', sep=';')

### 1. Постройте список уникальных типов самолетов зарегистрированных в России

In [3]:
unique_planes = aircraft_df[aircraft_df['Опознавательные знаки'].str.startswith('RA') &
                        (aircraft_df['Вид воздушного судна'] == 'самолет')]
np.unique(unique_planes['Тип (наименование) воздушного судна'])

array(['"ZODIAK" CH-701 STOL', '"АВИА"-АЗЦ-14', '"Форсаж"', ...,
       'Яр-12 "Ворон"', 'Ястреб', 'Яша'], dtype=object)

### 2. Какой тип самолета имеет самую раннюю дату выдачи сертификата?


In [4]:
times_str = aircraft_df['дата действующего свидетельства о регистрации'].values

def string_to_datetime(string):
        if string:
            d, m, y = tuple(string.split('.'))
            return datetime.date(int(y), int(m), int(d))

times_datatime = list(map(string_to_datetime, times_str))
time_list = str(min(times_datatime)).split('-')
time_list.reverse()
time_str = '.'.join(time_list)

oldest_plane = aircraft_df.iloc[list(times_str == time_str).index(True), 1]
oldest_plane

'Ан-2'

### 3. Постройте запрос: Владелец аэропорта, Аэропорт, Пассажиропоток суммарный за 2018 год, Грузопоток суммарный за 2018 год

In [17]:
airports_df['Наименование аэропорта РФ'] =\
airports_df['аэропорт'].astype('str').apply(lambda x: x.strip().split()[0])

In [6]:
passenger_df_ = passenger_df.drop(columns=['Январь - Декабрь'])
passenger_df_.replace('***', 0, inplace=True)
passenger_df_.fillna(0, inplace=True)

for column in passenger_df_.columns[1:]:
    passenger_df_[column] = passenger_df_[column].astype('str').apply(lambda x: int(x.replace(' ', '')))
passenger_df_['Суммарный пассажиропоток'] = passenger_df_.iloc[:, 2:].sum(axis=1).copy()

In [7]:
cargo_df_ = cargo_df.replace('***', 0)
cargo_df_['Год периода данных'] = cargo_df_['Год периода данных'].apply(lambda x: int(x.replace(' ', '')))
cargo_df_ = cargo_df_.drop(columns=['Январь - Декабрь'])
cargo_df_.fillna(0, inplace=True)

for column in cargo_df_.columns[2:]:
    cargo_df_[column] = cargo_df_[column].astype('str').apply(lambda x: float(x.replace(',', '.').replace(' ', '')))
cargo_df_['Суммарный грузопоток'] = cargo_df_.iloc[:, 2:].sum(axis=1).copy()

In [8]:
flow_df = passenger_df_[['Наименование аэропорта РФ', 'Суммарный пассажиропоток']]
flow_df['Суммарный грузопоток'] = cargo_df_['Суммарный грузопоток']
flow_df = flow_df.iloc[np.where(passenger_df_['Год периода данных'] == 2018)[0]]
flow_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Наименование аэропорта РФ,Суммарный пассажиропоток,Суммарный грузопоток
0,Абакан,150533,638.50
1,Айхал,0,0.00
2,Алдан,0,0.00
3,Амдерма,0,0.00
4,Анадырь,78826,1492.64
...,...,...,...
256,Якутск,707508,7055.46
257,Якутск(Маган),0,0.00
258,Ямбург,45828,1.66
259,Ярославль(Туношна),18935,1933.63


### 4. Перечислите аэропорты, где пассажиропоток меньше медианы, а грузопоток больше медианы 

In [9]:
passengers_df = pd.read_csv('passenger transportation.csv', sep=';')
passengers_df = passengers_df.where(~(passengers_df == '***'), None)
passengers_df = passengers_df.where(~pd.isna(passengers_df), None)

In [10]:
cargo_df = pd.read_csv('cargo transportation.csv', sep=';')
cargo_df = cargo_df.where(~(cargo_df == '***'), None)
cargo_df = cargo_df.where(~pd.isna(cargo_df), None)

In [11]:
# вычислим медиану пассажиропотока и грузооборота
passengers_np = passengers_df.iloc[:, 2:-1].values.reshape(-1)
passengers_np = np.delete(passengers_np, passengers_np == None)

passengers_ints = sorted(list(map(lambda x: int(x.replace(' ', '')), passengers_np)))
passengers_median = passengers_ints[int(len(passengers_ints)/2)]

cargo_np = cargo_df.iloc[:, 2:-1].values.reshape(-1)
cargo_np = np.delete(cargo_np, cargo_np == None)

cargo_ints = sorted(list(map(lambda x: float(x.replace(',', '.').replace(' ', '')), cargo_np)))
cargo_median = cargo_ints[int(len(cargo_ints)/2)]

# passengers_median, cargo_median

У каждого аэропорта грузопоток и пассажиропоток будем считать средними по известным значениям за год

In [12]:
airports = cargo_df.iloc[:, 0]
def get_mean(x):
    x = np.delete(x, x == None)
    g = lambda y: float(y.replace(' ', '').replace(',', '.'))
    p = list(map(g, x))
    if len(p) == 0:
        return None
    else:
        return np.mean(p)

passengers_means = np.array(list(map(get_mean, passengers_df.iloc[:, 2:-1].values)))
cargo_means = np.array(list(map(get_mean, cargo_df.iloc[:, 2:-1].values)))

def passengers_compare(p):
    if p and passengers_median:
        return p < passengers_median
    else:
        return False

def cargo_compare(c):
    if c and cargo_median:
        return c > cargo_median
    else:
        return False

passengers_comparison = np.array(list(map(passengers_compare, passengers_means)))
cargo_comparison = np.array(list(map(cargo_compare, cargo_means)))
and_comparison = passengers_comparison * cargo_comparison

airports.values[and_comparison]

array(['Анадырь', 'Мирный', 'Ярославль(Туношна)', 'Анадырь',
       'Комсомольск-на-Амуре', 'Мирный', 'Хатанга', 'Чита(Кадала)',
       'Ярославль(Туношна)', 'Анадырь', 'Благовещенск', 'Кемерово',
       'Комсомольск-на-Амуре', 'Магадан(Сокол)', 'Мирный', 'Хатанга',
       'Чита(Кадала)', 'Ярославль(Туношна)', 'Анадырь', 'Благовещенск',
       'Кемерово', 'Комсомольск-на-Амуре', 'Магадан(Сокол)', 'Мирный',
       'Чита(Кадала)', 'Ярославль(Туношна)', 'Анадырь', 'Благовещенск',
       'Комсомольск-на-Амуре', 'Магадан(Сокол)', 'Мирный', 'Хатанга',
       'Чита(Кадала)', 'Ярославль(Туношна)', 'Анадырь', 'Благовещенск',
       'Комсомольск-на-Амуре', 'Магадан(Сокол)', 'Мирный', 'Хатанга',
       'Чита(Кадала)', 'Ярославль(Туношна)', 'Анадырь', 'Благовещенск',
       'Комсомольск-на-Амуре', 'Ленск', 'Магадан(Сокол)', 'Мирный',
       'Певек', 'Полярный', 'Хатанга', 'Чита(Кадала)',
       'Ярославль(Туношна)', 'Анадырь', 'Кемерово', 'Магадан(Сокол)',
       'Мирный', 'Хатанга', 'Ярославль(Ту


### 5. • Перечислите авиакомпании у которых нет типов воздушных судов зарегистрированных в России 


In [13]:
def plane_check(plane_string):
    plane_string_processed = []
    for plane_name in plane_string.split(','):
        plane_string_processed.append(plane_name[:-4].strip())
    for plane in plane_string_processed:
        if plane in airlines_df['типы воздушных судов'].values:
            return False
        else:
            continue
    return True

airlines_df_ = airlines_df
airlines_df_['все самолеты (не РФ)'] =\
    airlines_df_[airlines_df_["типы воздушных судов"].notna()]['типы воздушных судов'].\
    astype('str').apply(plane_check)

airlines_df_[airlines_df_['все самолеты (не РФ)'] == True]['короткое название'].values

array(['Абакан Эир', 'Авиа Менеджмент Груп', 'Авиакомпания «Лайт Эйр»',
       'Авиакомпания СОКОЛ', 'Авиакон Цитотранс', 'Авиалифт Владивосток',
       'Авиасервис (Казань) (ИНН 1624446694)', 'Авиастар-ТУ', 'Авиашельф',
       'Аврора', 'Ай Флай', 'АЛРОСА Авиакомпания ЗАО',
       'Алтайские Авиалинии', 'Амур', 'Амурская авиабаза',
       'Ангара АК ЗАО', 'Арго', 'Арт Авиа', 'АТРАН', 'АэроБратск',
       'АэроГео', 'АЭРОКУЗБАСС', 'Аэролимузин', 'Аэропорт Оренбург',
       'Аэросервис (Даг.)', 'Аэросервис (Чита)', 'Аэрофлот ОАО',
       'Балтийские авиалинии', 'БАРКОЛ', 'Бурал', 'Вертикаль-Т ООО',
       'ВЗЛЕТ НПО, ЗАО', 'ВИМ-АВИА', 'ВИТЯЗЬ-АЭРО АК ООО', 'ВОЛГА-ДНЕПР',
       'Вологодское АП', 'Восток', 'Газпром авиа ООО', 'Геликс', 'Глобус',
       'Грозный Авиа ОАО', 'Дальнереченск Авиа', 'Дельта К',
       'Джет Эйр Групп', 'ДОНАВИА', 'Ельцовка АП ЗАО', 'Ижавиа ОАО',
       'Икар ООО', 'ИрАэро АК ЗАО', 'Казанское авиапредприятие',
       'Камчатское АП', 'Катэкавиа', 'Когалымавиа О

### 6. • Выведите список: Месяц, суммарный пассажиропоток за данный месяц, аэропорт в котором пассажиропоток в данном месяце максимальный 


In [14]:
passengers_df = pd.read_csv('passenger transportation.csv', sep=';')
passengers_df = passengers_df.where(~(passengers_df == '***'), None)
passengers_df = passengers_df.where(~pd.isna(passengers_df), None)
passengers_df.head()

,Наименование аэропорта РФ,Год периода данных,Январь,Февраль,Март,Апрель,Май,Июнь,Июль,Август,Сентябрь,Октябрь,Ноябрь,Декабрь,Январь - Декабрь
0,Абакан,2018,11 851,10 504,11 591,12 323,14 314,19 935,25 213,25 507,19 295,None,None,None,None
1,Айхал,2018,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Алдан,2018,None,None,None,None,None,None,None,None,None,None,None,None,None
3,Амдерма,2018,None,None,None,None,None,None,None,None,None,None,None,None,None
4,Анадырь,2018,5 128,4 447,5 456,7 650,8 364,10 563,13 402,13 751,10 065,None,None,None,None


In [15]:
def str_to_int(x):
    f = lambda y: int(y.replace(' ', '')) if y else y
    return x.apply(f)

pass_months_df = passengers_df.iloc[:, 2:-1].fillna(0)
pass_months_df = pass_months_df.apply(str_to_int)
sum_months_df = pass_months_df.apply(sum)

data = {}
months = passengers_df.iloc[:, 2:-1].columns.values
pass_max = pass_months_df.apply(np.argmax).values
for month, idx in zip(months, pass_max):
    data[month] = passengers_df.iloc[idx, 0]
final_df = pd.DataFrame(pd.Series(data), 
                        columns=['аэропорт с максимальным пассажиропотоком'])
final_df['суммарный пассажиропоток'] = sum_months_df
final_df

,аэропорт с максимальным пассажиропотоком,суммарный пассажиропоток
Январь,Москва(Шереметьево),92581956
Февраль,Москва(Шереметьево),84279197
Март,Москва(Шереметьево),97548315
Апрель,Москва(Шереметьево),103999255
Май,Москва(Шереметьево),120740351
Июнь,Москва(Шереметьево),146081971
Июль,Москва(Шереметьево),167043305
Август,Москва(Шереметьево),171378013
Сентябрь,Москва(Шереметьево),148372441
Октябрь,Москва(Шереметьево),108301174


### 7. • Выведите список: Тип аэропорта, средний грузопоток в месяц в аэропортах данного типа

In [16]:
data = {
    'Наименование аэропорта РФ': cargo_df.iloc[:, 0].values,
    'средний грузопоток': cargo_means,
}
pd.DataFrame(data)

,Наименование аэропорта РФ,средний грузопоток
0,Абакан,70.9444
1,Айхал,None
2,Алдан,None
3,Амдерма,None
4,Анадырь,165.849
...,...,...
3468,Якутск,1131.45
3469,Якутск(Маган),None
3470,Ямбург,4.6325
3471,Ярославль(Туношна),1271.91
